# Spatial discretisation

So far, we've seen time derivatives and ordinary differential equations of the form

$$
\dot{u} = f(t, u).
$$

Most problems one encounters in the real world have spatial as well as time derivatives. Our first example is the [*Poisson equation*](https://en.wikipedia.org/wiki/Poisson%27s_equation):

$$
\begin{align}
-\frac{\text{d}^2 u}{\text{d} x^2} &= f(x) \quad x \in \Omega = (-1, 1)\\
 u(-1) &= a\\
 \frac{\text{d} u}{\text{d} x}(1) &= b\\
 \end{align}.
$$

This is termed a *boundary value problem* (BVP), as opposed to the ODEs which were *initial value problems*, since we do not specify an initial condition, but rather a condition on the boundary of the domain.

This equation appears in a remarkably large number of places. As one example, it models the equilibrium temperature profile of a thermally conducting material maintained at constant temperature $a$ on the left and cooled at constant rate $b$ on the right.

To solve this problem numerically we must make a number of choices:

- how to represent the solution $u$;
- how to compute its derivatives;
- how to enforce the boundary conditions;
- how and where to evaluate $f$;
- in what sense we would like our solution to satisfy the equation.

[Iserles' book](http://www.damtp.cam.ac.uk/user/ai/Arieh_Iserles/Textbook.html) contains a quite mathematical treatment of finite differences. I also like [Randy LeVeque's *Finite difference methods for ordinary and partial differential equations*](http://staff.washington.edu/rjl/fdmbook/).

## Finite difference framework

We will focus on _finite difference_ methods here, which make the following choices in answer to the questions above:

- The solution $u(x)$ is represented by _pointwise_ values $u_i = u(x_i)$ at some discrete set of points $-1 = x_0 < x_1 < \dots < x_N = 1$. Importantly, the framework _does not_ specify the value of $u$ outside of these points;
- derivatives of $u$ at points $x_i$ are approximated using differencing formulae that utilise a finite number of neighbouring points (independent of $N$);
- boundary conditions are either enforced pointwise (e.g. the $u(-1)$ case above), or (when constraining derivatives) with one-sided differencing formulae;
- $f$ is evaluated pointwise at each $x_i$;
- we require that our finite difference method satisfies the equation pointwise at each $x_i$ in the interior of the domain.

### Differencing formulae

Our starting point is the definition of a derivative:

$$
\frac{d u(x)}{d x} = \lim_{\epsilon \to 0} \frac{u(x + \epsilon) - u(x)}{\epsilon}.
$$

If we wish to approximate this in our finite difference framework, where we only have point values, we can do so using neighbouring values. Writing $x_{i+1} - x_i = h$ for convenience, we can write

$$
\frac{d u(x_i)}{d x} \approx \frac{u(x_{i+1}) - u(x_i)}{h} = \frac{u_{i+1} - u_i}{h} =: D_+ u_i.
$$

This is a _one-sided_ approximation: we only use $u_i$ and $u_{i+1}$. Another one-sided approximation would be to offset in the other direction

$$
D_{-} u_i := \frac{u_i - u_{i-1}}{h}.
$$

Finally, we can also use a _centred_ approximation by averaging the two one-sided approximations:

$$
D_0 u_i := \frac{u_{i+1} - u_{i-1}}{2h} = \frac{1}{2} (D_+ + D_-) u_i.
$$

Let's have a picture

In [ ]:
%matplotlib notebook

import numpy
from matplotlib import pyplot
import matplotlib.lines as mlines
pyplot.style.use('ggplot')

n = 200
h = 2/(n-1)
x = numpy.linspace(1,2.5,n)
pyplot.plot(x, numpy.sin(x));

def newline(p1, p2, **kwargs):
    ax = pyplot.gca()
    xmin, xmax = ax.get_xbound()

    if(p2[0] == p1[0]):
        xmin = xmax = p1[0]
        ymin, ymax = ax.get_ybound()
    else:
        ymax = p1[1]+(p2[1]-p1[1])/(p2[0]-p1[0])*(xmax-p1[0])
        ymin = p1[1]+(p2[1]-p1[1])/(p2[0]-p1[0])*(xmin-p1[0])

    l = mlines.Line2D([xmin,xmax], [ymin,ymax], **kwargs)
    ax.add_line(l)
    return l

h = 0.25
xi = 1.6
ximinus = xi - h
xiplus = xi + h

pyplot.plot([ximinus, xi, xiplus], numpy.sin([ximinus, xi, xiplus]), marker="o", linestyle="none")

newline((xi, numpy.sin(xi)), (xiplus, numpy.sin(xiplus)), linestyle="dashed", label="$D_+ u(x)$")
newline((xi, numpy.sin(xi)), (ximinus, numpy.sin(ximinus)), linestyle="dotted", label="$D_- u(x)$")
newline((ximinus, numpy.sin(ximinus)), (xiplus, numpy.sin(xiplus)), linestyle="-.", label="$D_0 u(x)$")


newline((xi, numpy.sin(xi)), (xiplus, numpy.sin(xi) + h*numpy.cos(xi)), color="black", label="$u'(x)$")

pyplot.legend();

Let's look at the full approximate derivative too.

In [ ]:
n = 10
h = 2/(n-1)
x = numpy.linspace(-1,1,n)
u = numpy.sin(x)
pyplot.figure()
pyplot.plot(x, numpy.cos(x), label="$u'$");
pyplot.plot(x[:-1], (u[1:] - u[:-1])/h, label="$D_+$", marker="o", linestyle="none")
pyplot.legend();

## Accuracy

Certainly from the picture of the slope above, it appears that the centered difference formula is more accurate than the one-sided approximations. Can we formalise this at all?

To do so, we turn to the favourite tool of the budding numericist, the *taylor expansion*.

### Recap, Taylor expansions

For a sufficiently smooth function $u$, given a point $x$, we can represent the function at a new point $x + h$ by its Taylor expansion

$$
u(x + h) = u(x) + u'(x) h + \frac{1}{2} u''(x) h^2 + \frac{1}{6} u'''(x) h^3 + \dots = \sum_{n=0}^\infty \frac{1}{n!} h^n u^{(n)}(x) 
$$

where the notation $u'$ is shorthand for $\frac{\text{d} u}{\text{d} x}$ and $u^{(n)}(x) = \frac{\text{d}^n u}{\text{d} x^n}$.

Let's look at how this works for a sample function.

In [ ]:
from functools import reduce
from operator import mul

def u(x, n=0):
    factor = (-1)**(n // 2)
    if n % 2 == 0:
        u_ = numpy.sin
    else:
        u_ = numpy.cos
    return factor * u_(x)

pyplot.figure()

x = numpy.linspace(0.5, 1.75, 500)

pyplot.plot(x, u(x));

x0 = 0.6
h = 0.8

def fac(n):
    return reduce(mul, range(1, n+1), 1)

def taylor(u, x0, h, n):
    return u(x0) + sum(h**i/fac(i) * u(x0, i) for i in range(1, n))

xs = numpy.linspace(x0, x0+h, 20)
for n in range(1, 5):
    pyplot.plot(xs, [taylor(u, x0, x - x0, n) for x in xs], marker="o", label=r"$\tilde{u} + \mathcal{O}(h^%d)$" % n)

pyplot.legend();

If we chop off the series at some finite $n$ we write

$$
u(x + h) = u(x) + u'(x) h + \frac{1}{2} u''(x) h^2 + \mathcal{O}(h^3)
$$

with $h$ sufficiently small.

To determine the order of a method, we substitute the Taylor expansion into the differencing expression and calculate.

As an example, let us consider the one-sided differencing operator $D_+$. To simplify notation, we will choose $x = 0$, and we have

$$
\begin{align}
u'(0) &\approx \frac{u(h) - u(0)}{h} \quad \text{ definition of } D_+\\
      &= h^{-1}(\underbrace{u(0) + u'(0) h + \frac{1}{2} u''(0) h^2 + \mathcal{O}(h^3)}_{u(h)} - u(0)) \\
      &= u'(0) + \frac{1}{2} u''(0) h + \mathcal{O}(h^2)\\
\end{align}.
$$

The leading-order error term in the right hand side is $\mathcal{O}(h)$, and so we say that this is a _first-order_ method. Derivation that the operator $D_-$ is also first-order proceeds identically.

#### Questions

1. Show that the centered difference operator $D_0$ computes a second-order accurate derivative.

## Stability

We will postpone mathematical discussion of stability for a while, and give an intuition for some potential problems. Let us first check that our implementation of differencing operators provides us with the expected (mathematical) convergence orders for a smooth function.

In [ ]:
def dplus(x, u):
    return x[:-1], (u[1:] - u[:-1])/(x[1:] - x[:-1])

def dminus(x, u):
    return x[1:], (u[1:] - u[:-1])/(x[1:] - x[:-1])

def center(x, u):
    return x[1:-1], (u[2:] - u[:-2])/(x[2:] - x[:-2])

In [ ]:
grids = 2**numpy.arange(3, 10)

def error(f, df, op):
    for n in grids:
        x = numpy.linspace(-1, 1, n)
        x, y = op(x, f(x))
        yield numpy.linalg.norm(y - df(x), numpy.inf)

pyplot.figure()
for op in [dplus, dminus, center]:
    pyplot.loglog(1/grids, list(error(numpy.sin, numpy.cos, op)), marker="o", linestyle="none", label=op.__name__)
    
pyplot.xlabel("Resolution ($h$)")
pyplot.ylabel("$l_\infty$ error in derivative")

pyplot.loglog(1/grids, 1/grids, label="$h$")
pyplot.loglog(1/grids, 1/grids**2, label="$h^2$")
pyplot.legend();

So both the one-sided differences are first-order accurate, whereas the centered difference is second-order accurate, as expected. One thing to be wary of, however, is using some of these approximations for functions that are "rough" on the grid scale.

We can make this question more precise by asking whether there are functions whose derivatives are non-zero, but for which our numerical approximations compute $u'(x_i) = 0$.

Let's try and contrive an example:

In [ ]:
x = numpy.linspace(-1, 1, 9)
xf = numpy.linspace(-1, 1, 100)

def f(x):
    return numpy.cos(1/2 + 4*numpy.pi*x)

def df(x):
    return -4*numpy.pi*numpy.sin(1/2 + 4*numpy.pi*x)

In [ ]:
pyplot.figure()
pyplot.plot(x, f(x), marker="o", label="coarse")
pyplot.plot(xf, f(xf), "-", label="fine")
pyplot.legend();

What about the derivatives?

In [ ]:
pyplot.figure()
for op in [dplus, dminus, center]:
    x_, y = op(x, f(x))
    pyplot.plot(x_, y, "o-", label=op.__name__)
pyplot.plot(xf, df(xf), "-", label="Exact")
pyplot.legend();

The centered difference approximation produces a _zero_ derivative for this function. Hence if we have a solution $u(x)$, we can (at least to the numerical operator) construct a new solution $\tilde{u}(x) = u(x) + f(x)$.

Suddenly, even if our actual equation has a unique solution, the numerical solution is no longer unique. This turns out to cause all kinds of terrible issues with numerical algorithms and must be avoided at all costs.

## Higher-order derivatives

We can compute high-order derivatives by repeatedly applying differencing operators for lower-order derivatives.

For example, the second derivative

$$
\frac{\text{d}^2 u}{\text{d} x^2} \approx D^2 u_i = D_+ D_- u_i = \frac{1}{h^2}\left(u_{i+1} - 2 u_i + u_{i-1}\right) = D_- D_+ u_i.
$$

### Questions

1. Show that this is a _second-order_ accurate approximation of the second derivative
2. We could also use $D^2 u_i = D_0 D_0 u_i$, derive the stencil for this case.
3. Finally, show that if we define a "half-step" centered difference operator

$$
\hat{D}_0 u = \frac{1}{h}\left[u\left(x + \frac{h}{2}\right) - u\left(x - \frac{h}{2}\right)\right]
$$
then we have

$$
D^2 = \hat{D}_0 \circ \hat{D}_0 = D_+ \circ D_-
$$

## Boundary conditions

The final missing piece required before we can solve the our first PDE is to figure out how we will treat boundary conditions. To do this, we will first recast the differencing operators as matrices. it is then somewhat easier to see what is going on.

We can think of the differencing operator acting on an entire vector

$$
U = \begin{bmatrix}
u_0\\
u_1\\
\vdots\\
u_N
\end{bmatrix}
$$

at once. For example, we can write

$$
D_+ = \begin{bmatrix}
-1 & 1 & 0 & \dots & 0\\
0 & -1 & 1 & \dots & 0\\
\vdots & \ddots & \ddots & \vdots & \vdots\\
0 & \dots & 0 & -1 & 1\\
0 & \dots & 0 & 0 & -1\\
\end{bmatrix}
$$

and

$$
D^2 = \begin{bmatrix}
-2 & 1 & 0 &  \dots & 0\\
1 & -2 & 1 &  \dots & 0\\
\vdots &  & \ddots &  & \vdots\\
0 & \dots & 0 & 1 & -2\\
\end{bmatrix}.
$$

Recall, our problem was to find $u \in (-1, 1)$ satisfying

$$
\begin{align}
\frac{\text{d}^2 u}{\text{d} x^2} u &= f\\
u(-1) &= a\\
\frac{\text{d} u}{\text{d} x}(1) &= b
\end{align}
$$

In matrix form, this becomes

$$
\underbrace{D^2}_{A} \underbrace{\begin{bmatrix}
u_0\\
u_1\\
\vdots\\
u_N
\end{bmatrix}}_{U} = \underbrace{\begin{bmatrix} f_0\\f_1\\\vdots\\f_N\end{bmatrix}}_{F}.
$$

This works perfectly in the interior of the domain, but we need to figure out what to do at the boundaries. For example, we can't use the standard differencing operator for $D^2$ on $u_N$, because $u_{N+1}$ does not exist.
Fortunately, our boundary conditions inform how to modify the matrix appropriately.

### Dirichlet conditions

These conditions, of the form 

$$
u(-1) = a
$$

specify the _value_ of the solution at a particular point (or set of points). This means, that rather than solving a small equation to determine the value at this point, we _already know_ and can instead replace the relevant rows of the matrix. Let us suppose we have ordered our points such that $u(-1)$ corresponds to $u_0$. Then we have

$$
\begin{bmatrix}
1 & 0 & \dots & 0\\
 & & & \\
 & & A_{1:,:} & \\
 & & &\\
\end{bmatrix}
U = \begin{bmatrix} a\\ \\ F_{1:} \\ \\ \end{bmatrix}.
$$

In general, if we have a boundary value $\alpha_i$ that constrains $u_i$ then we replace the $i$th row with the identity, and the $i$th value in the right hand side with $\alpha_i$.

This modification destroys any symmetry that might have existed in the matrix $A$, since we have zeroed rows, but not the corresponding columns. If we write the linear system in block form, we can, however, see a way around this:

$$
\begin{bmatrix}
I & 0\\
A_{10} & A_{11}
\end{bmatrix}
\begin{bmatrix}
U_0\\
U_1
\end{bmatrix}
=
\begin{bmatrix}
F_0\\
F_1
\end{bmatrix}
$$

Since we know $U_0$ (they are just $F_0$), we can forward-substitute and move the lower-left block of the matrix onto the right hand side, to produce

$$
\begin{bmatrix}
I & 0\\
0 & A_{11}
\end{bmatrix}
\begin{bmatrix}
U_0\\
U_1
\end{bmatrix}
=
\begin{bmatrix}
F_0\\
F_1 - A_{10}F_0
\end{bmatrix}.
$$

This is often a convenient form to work with.